In [1]:
#################   Yu's ##########################
#--> Yu's < from clamp.dataset import InMemoryClamp
#--> Yu's < from clamp import utils
#--> Yu's < from clamp.utils import set_device

#################   Yu's ##########################


import os
import pandas as pd
from loguru import logger
import random
import torch
import numpy as np

import mlflow
import argparse
import wandb
from time import time

from loguru import logger

from pathlib import Path

/storage/homefs/yc24j783/miniconda3/envs/pyg/lib/python3.9/site-packages/mlflow/protos/service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


In [ ]:
"""example call:
python clamp/train.py \
    --dataset=./data/fsmol \
    --split=FSMOL_split \
    --assay_mode=clip \
    --compound_mode=morganc+rdkc 
"""

""" training pubchem23 without downstream datasets
python clamp/train.py \
    --dataset=./data/pubchem23/ \
    --split=time_a \
    --assay_mode=clip \
    --batch_size=8192 \
    --dropout_hidde=0.3 \
    --drop_cidx_path=./data/pubchem23/cidx_overlap_moleculenet.npy \
    --train_subsample=10e6 \
    --wandb \
    --experiment=pretrain
"""

In [ ]:
def parse_args_override(override_hpjson=True):
    parser = argparse.ArgumentParser('Train and test a single run of clamp-Activity model. Overrides arguments from hyperparam-file')
    parser.add_argument('-f', type=str)
    #--> Yu's < default='./data/fsmol/'
    parser.add_argument('--dataset', type=str, default='./data/fsmol/', help='Path to a prepared dataset directory.')
    #--> Yu's < assay_features (clip trained by GPCR assay corups)
    parser.add_argument('--assay_mode', type=str, default='lsa', help='Type of assay features ("clip", "biobert" or "lsa").')
    #--> Yu's < default='morganc+rdkc' --> SMILESTokenizer
    parser.add_argument('--compound_mode', type=str, default='morganc+rdkc', help='Type of compound features (default: morganc+rdkc)')
    #--> Yu's < default='.hparams/default.json'
    parser.add_argument('--hyperparams',type=str, default='./hparams/default.json', help='Path to hyperparameters to use in training (json, Hyperparams, or logs).')
    
    parser.add_argument('--checkpoint', help='Path to a model-optimizer PyTorch checkpoint from which to resume training.', metavar='')
    parser.add_argument('--experiment', type=str, default='debug', help='Name of MLflow experiment where to assign this run.', metavar='')
    parser.add_argument('--random', action='store_true', help='Forget about the specified model and run a random baseline.')

    #parser.add_argument('--optimizer', type=str, default='AdamW', help='Optimizer to use for training (default: AdamW).')
    #parser.add_argument('--l2', type=float, default=0.01, help='Weight decay to use for training (default: 0.01).')
    #parser.add_argument('--loss_fun', type=str, default='BCE', help='Loss function to use for training (default: BCE).')
    #parser.add_argument('--epoch_max', type=int, default=50, help='Maximum number of epochs to train for (default: 100).')
    #parser.add_argument('--lr_ini', type=float, default=1e-5, help='Initial learning rate (default: 1e-5).')
    
    parser.add_argument('--gpu', type=str, default="0", help='GPU number to use. Default: 0', metavar='')
    parser.add_argument('--seed', type=int, default=None, help='seed everything with provided seed, default no seed')

    parser.add_argument('--split',type=str, default='time_a_c',help="split-type default: time_a_c for time based assay and compound split, other options time_a, time_c, random_{seed}, or column of activity.parquet triplet")
    parser.add_argument('--support_set_size',type=int, default='0',help="per task how many to add from test- as well as valid- to the train-set default=0 = zero-shot")
    parser.add_argument('--train_only_actives', action='store_true', help='train only with active molecules')
    parser.add_argument('--drop_cidx_path', type=str, default=None, help='Path to a file containing a np of cidx (NOT CIDs) to drop from the dataset.')

    parser.add_argument('--verbose','-v', type=int, default=0, help='verbosity level default=0')
    parser.add_argument('--wandb','-w', action='store_true', help='wandb logging on')
    parser.add_argument('--bf16', action='store_true', help='use bfloat16 for training')
    
    args, unknown = parser.parse_known_args()
    keypairs = dict([unknown[i:i+2] for i in range(0, len(unknown), 1) if unknown[i].startswith("--") and not (unknown[i+1:i+2]+["--"])[0].startswith("--")])

    hparams = utils.get_hparams(path=args.hyperparams, mode='json', verbose=args.verbose)
    
    if override_hpjson:
        from clamp.utils import NAME2FORMATTER
        for k,v in NAME2FORMATTER.items():
            if (k not in args):
                default = hparams.get(k, None)
                parser.add_argument('--'+k, type=v, default=default)
                if (k in keypairs):
                    logger.info(f'{k} from hparam file will be overwritten')

        args = parser.parse_args()

    if args.nonlinearity is None:
        args.nonlinearity = 'ReLU'
    if args.compound_layer_sizes is None:
        logger.info('no compound_layer_sizes provided, setting to hidden_layers')
        args.compound_layer_sizes = args.hidden_layers
    if args.assay_layer_sizes is None:
        logger.info('no assay_layer_sizes provided, setting to hidden_layers')
        args.assay_layer_sizes =  args.hidden_layers

    return args
